In [15]:
import torch
import torch.nn as nn 
import torchvision
from torchvision import datasets, transforms, models

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import io
import requests
import time
import json

In [17]:
device = "cpu" # I'm not sure if it will work out of the box with cuda even if you have a GPU
model = models.vgg16(pretrained=True)
full_model = nn.Sequential(model, nn.Softmax(dim=1)).to(device).eval()

# Label Conversion

This step is done because the labels for the validation dataset do not match the labels from the pretrained model. 

In [22]:
PATH = 'labels/imagenet_class_index.json'
with open(PATH, 'r') as fp:
    labels_65 = json.load(fp)

# labels_65 is the default labels from pretrained model
labels_65 = {value[0]: int(key) for key,value in labels_65.items()} 

labels_file = open('labels/labels.txt')
labels_490 = {} #labels_490 is the labels from the validation dataset txt file
for line in labels_file:
    contents = line.split()
    val = contents[2]
    key = contents[1]
    n_name = contents[0] 
    labels_490.update({int(key): [n_name,val]})

In [23]:
def Convert_labels_file(k_490):
    n_name = labels_490[k_490][0]
    k_65 = labels_65[n_name]
    
    return k_65

In [20]:
ground_truth_handle = open('labels/ILSVRC2012_validation_ground_truth.txt')

count = 0
num_images = 1000
Converted_labels = []
for line in ground_truth_handle:
    if count == num_images:
        break
    k_65 = Convert_labels_file(int(line))
    Converted_labels.append(k_65)
    
    count+=1

In [21]:
min_img_size = 224 
fro = 150
imgs = 100
cls = {}
for i in range(fro,fro+imgs):
    if(i<10):
        directory_imgs = 'ILSVRC2012_img_val/ILSVRC2012_val_00000' +'00' +str(i) 
    elif(i<100):
        directory_imgs = 'ILSVRC2012_img_val/ILSVRC2012_val_00000' +'0' +str(i)
    elif(i<1000):
        directory_imgs = 'ILSVRC2012_img_val/ILSVRC2012_val_00000' + str(i)
        
    filename = directory_imgs 
    img = Image.open(directory_imgs+'.JPEG')
    pre_transform = transforms.Compose([transforms.Resize((min_img_size,min_img_size))])
    img = pre_transform(img)
    original = transforms.ToTensor()(img).to(device)[None,]
    original_class = model(original).detach().cpu().numpy().argmax()
    cls.update({i:original_class})
    
    print(i,original_class,Converted_labels[i-1])

150 977 975
151 617 620
152 637 637
153 39 39
154 107 115
155 108 937
156 273 272
157 277 277
158 763 763
159 905 789
160 646 646
161 207 213
162 894 493
163 438 647
164 504 504
165 937 937
166 687 687
167 781 781
168 666 666
169 583 583
170 158 158
171 884 825
172 212 212
173 659 659
174 257 257
175 436 436
176 199 196
177 140 140
178 250 248
179 728 339
180 230 230
181 999 361
182 659 544
183 961 935
184 639 638
185 627 627
186 434 289
187 908 867
188 270 272
189 103 103
190 584 584
191 208 180
192 448 703
193 510 449
194 771 771
195 118 118
196 396 396
197 470 934
198 129 16
199 556 548
200 8 993
201 861 704
202 452 457
203 233 233
204 416 401
205 605 827
206 6 376
207 363 146
208 606 606
209 922 922
210 516 516
211 284 284
212 731 889
213 475 475
214 419 978
215 861 475
216 984 984
217 16 16
218 77 77
219 678 610
220 87 254
221 636 636
222 662 662
223 473 473
224 220 213
225 28 25
226 720 463
227 212 215
228 250 173
229 305 35
230 741 741
231 913 125
232 913 787
233 289 289
234 425

In [392]:
correct = 0
for i in range(fro,fro+imgs):
    predicted_val = cls[i]
    true_val = Converted_labels[i-1]
    if true_val == predicted_val:
        correct+=1
accuracy = correct/(imgs+1)
print(accuracy)
    

0.43564356435643564


In [26]:
def __fgsm(model, x, label, epsilon, targeted, clamp):
    model.zero_grad()

    x = torch.as_tensor(x, device=device)
    x.requires_grad = True
    
    logits = model(x)
    target = torch.LongTensor([label]).to(device)
    loss = nn.CrossEntropyLoss()(logits, target)
    loss.backward()
    
    diff = epsilon * x.grad.sign()
    
    if targeted:
        adv = x - diff
    else:
        adv = x + diff
    
    return adv.clamp(0, 1) if clamp else adv

def fgsm_targeted(model, x, label, epsilon, clamp=True):
    """
    model = neural network with logits output (not softmax)
    x = image 
    label = target label (don't care about the true label)
    epsilon = distance to move 
    clamp = Limit output values to the range [0,1]
    """
    return __fgsm(model, x, label, epsilon, True, clamp)
    
def fgsm_untargeted(model, x, label, epsilon, clamp=True):    
    """
    model = neural network with logits output (not softmax)
    x = image
    label = the true label
    epsilon = distance to move 
    clamp = Limit output values to the range [0,1]
    """
    return __fgsm(model, x, label, epsilon, False, clamp)

def plotter(image, adverserial, full_model):
    """Show the original (img), adversarial attack (adv), and some more stuff. Pass images as pytorch tensors"""
    image = image.cpu().detach()
    adverserial = adverserial.cpu().detach()
    
    probability_original = full_model(image.to(device)).cpu().detach().numpy()
    origial_image = image.numpy()[0]
    
    probability_adverserial = full_model(adverserial.to(device)).cpu().detach().numpy()
    adverserial_image = adverserial.numpy()[0]
    
    f, axes = plt.subplots(1,3, figsize=(15, 6))
    
    ax = axes[0]
    prediction = probability_original.argmax()
    ax.set_title("Original, class: {} ({:.0f} %): '{}'".format(
        prediction, 100*probability_original[0, prediction], labels[prediction])
    )
    ax.imshow(np.transpose(origial_image, (1,2,0)))
    
    ax = axes[1]
    prediction = probability_adverserial.argmax()
    ax.set_title("Adverserial, class: {} ({:.0f} %): '{}'".format(
        prediction, 100*probability_adverserial[0, prediction], labels[prediction])
    )
    ax.imshow(np.transpose(adverserial_image, (1,2,0)))
    
    ax = axes[2]
    diff = np.transpose(adverserial_image - origial_image, (1,2,0))
    plt.imshow(diff + 0.5)
    plt.show()

## Attack 100 validation images

Note: This will take some time.

In [28]:
min_img_size = 224 
fro = 150
imgs =  # Choose number of images you want to check
original_correct = 0
adv_correct = 0
cls = {}

for i in range(fro,fro+imgs):
    if(i<10):
        directory_imgs = 'ILSVRC2012_img_val/ILSVRC2012_val_00000' +'00' +str(i) 
    elif(i<100):
        directory_imgs = 'ILSVRC2012_img_val/ILSVRC2012_val_00000' +'0' +str(i)
    elif(i<1000):
        directory_imgs = 'ILSVRC2012_img_val/ILSVRC2012_val_00000' + str(i)
        
    filename = directory_imgs 
    img = Image.open(directory_imgs+'.JPEG')
    pre_transform = transforms.Compose([transforms.Resize((min_img_size,min_img_size))])
    img = pre_transform(img)
    original = transforms.ToTensor()(img).to(device)[None,]
    original_class = model(original).detach().cpu().numpy().argmax()
    cls.update({i:original_class})
    
    adv = fgsm_untargeted(model, original, label=original_class, epsilon=0.01, clamp=True)
    
    
    original = original.cpu().detach()
    adv = adv.cpu().detach()
    
    probability_original = full_model(original.to(device)).cpu().detach().numpy()
    origial_image = original.numpy()[0]
    
    probability_adverserial = full_model(adv.to(device)).cpu().detach().numpy()
    adverserial_image = adv.numpy()[0]
    
    prediction = probability_adverserial.argmax()
    
    print(i,Converted_labels[i-1],original_class, prediction)
    
    if(original_class == Converted_labels[i-1]):
        original_correct+=1
    if(prediction == Converted_labels[i-1]):
        adv_correct+=1

original_correct = original_correct/imgs*100 
adv_correct = adv_correct/imgs*100
print('Original_Accuracy: '+ str(original_correct) + ' %' )
print('Adversarial_Accuracy: '+ str(adv_correct) + str(' %'))

150 975 977 978
151 620 617 620
152 637 637 902
153 39 39 47
154 115 107 397
155 937 108 669
156 272 273 343
157 277 277 279
158 763 763 596
159 789 905 765
160 646 646 970
161 213 207 161
162 493 894 861
163 647 438 647
164 504 504 669
165 937 937 109
166 687 687 753
167 781 781 922
168 666 666 659
169 583 583 669
Original_Accuracy: 55.00000000000001 %
Adversarial_Accuracy: 10.0 %
